In [1]:
import sys
sys.path.append("../../../")

In [2]:
from onsager import crystal, supercell, cluster
import numpy as np
import scipy as sp
import itertools
import Transitions
import Cluster_Expansion
import MC_JIT
import time
from tqdm import tqdm
from numba import jit, int64, float64

## Set up the vector cluster expander

In [3]:
import pickle

# See if a cluster expander already exists
try:
    with open("ClusterExpander.pkl", "rb") as fl:
        print("Pre-created Cluster expander found")
        VclusExp = pickle.load(fl)

# Otherwise make one
except:
    N_units = 8 # No. of unit cells along each axis in the supercell
    NSpec = 3
    MaxOrder = 3
    MaxOrderTrans = 3

    a0 = 1.0
    cut = 1.01*a0*np.sqrt(3)/2  # Nearest neighbor cutoff

    crys = crystal.Crystal.BCC(a0, chemistry="A")
    jnetBCC = crys.jumpnetwork(0, cut)

    superlatt = N_units * np.eye(3, dtype=int)
    superBCC = supercell.ClusterSupercell(crys, superlatt)
    # Since the "spectator" tuple in the argument is left empty, all
    # sites of chemistry 0 will be mobile sites.

    vacsite = cluster.ClusterSite((0, 0), np.zeros(3, dtype=int))
    vacsiteInd = superBCC.index(np.zeros(3, dtype=int), (0, 0))[0]
    assert vacsiteInd == 0

    clusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrder)
    Tclusexp = cluster.makeclusters(crys, 1.01*a0, MaxOrderTrans)

    VclusExp = Cluster_Expansion.VectorClusterExpansion(superBCC, clusexp, Tclusexp, jnetBCC,
                                                             NSpec, vacsite, MaxOrder,
                                                             MaxOrderTrans, NoTrans=True)

    print("Done setting up")
    
    # save it for later
    with open("ClusterExpander.pkl", "wb") as fl:
        pickle.dump(VclusExp, fl)

Pre-created Cluster expander found


In [4]:
len(VclusExp.InteractVecInteracts), len(VclusExp.InteractVecVecs)

(5676, 5676)

In [5]:
# Check the correctness of the vectors - if they were assigned for the proper clusters
for interaction, vecInfo in VclusExp.Interact2VecInteract.items():
    # get the vectors for the interaction
    Intvecs = np.array([VclusExp.InteractVecVecs[info[0]][info[1]] for info in vecInfo])
    
    for classInd, interactInd in vecInfo:
        assert VclusExp.InteractVecInteracts[classInd][interactInd] == interaction
    
    # get the vectors for the representative cluster
    clRep = list(VclusExp.Interact2RepClustDict[interaction])[0]
    clVecs = np.array([VclusExp.vecVec[info[0]][info[1]] for info in VclusExp.clust2vecClus[clRep]])
    
    assert np.allclose(clVecs, Intvecs)
print("Test complete - vectors match with representative clusters")

Test complete - vectors match with representative clusters


In [6]:
# Next - match the clusters with the interactions
count = 0
for interaction, vecInfo in VclusExp.Interact2VecInteract.items():
    # get the vectors for the interaction
    Int0Sites = [(cluster.ClusterSite(ci=info[0], R=info[1]), spec) for info, spec in
                [(VclusExp.sup.ciR(siteInd), spec) for siteInd, spec in interaction]]
    
    siteList = [site for site, spec in Int0Sites]
    specList = [spec for site, spec in Int0Sites]
    
    repClust = Cluster_Expansion.ClusterSpecies(specList, siteList)
    siteSpecs = list(repClust.SiteSpecs)
    # Bring all sites within the unit cell and then translate
    siteListNew = [cluster.ClusterSite(R=site.R%8, ci=site.ci) for site,spec in siteSpecs]
    specListNew = [spec for site, spec in siteSpecs]
    repClust = Cluster_Expansion.ClusterSpecies(specListNew, siteListNew)
    
    # get the vectors for the representative cluster
    clRep = list(VclusExp.Interact2RepClustDict[interaction])[0]
    
    assert clRep == repClust, "\n count: {}\n From Interaction : {} \n Stored :\
    {} \n SitesOld:{} \n SiteNew: {} \n Specs: {}".format(count, repClust, clRep, siteList,
                                                              siteListNew, specList)
    
    count += 1
    
print("Test complete - representative clusters match with interactions")

Test complete - representative clusters match with interactions


In [7]:
# Next - get the dimensionality of the representative clusters
count = 0
for interaction, vecInfo in VclusExp.Interact2VecInteract.items():
    assert len(VclusExp.Interact2RepClustDict[interaction]) == 1
    # get the vectors for the representative cluster
    clRep = list(VclusExp.Interact2RepClustDict[interaction])[0]
    Glist = []
    for gop in VclusExp.crys.G:
        clNew = clRep.g(VclusExp.crys, gop)
        if clNew == clRep:
            Glist.append(gop.cartrot)
    
    G0 = sum(Glist)/len(Glist)
    
    vals, vecs = np.linalg.eig(G0)
    dim = 0
    for eigval in vals:
        if np.allclose(eigval, 1.0):
            dim += 1
    
    assert dim == len(vecInfo)
    
print("Test complete - dimensionality match found")

Test complete - dimensionality match found
